In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import glob
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
import joblib
import import_ipynb

from ipynb.fs.full.hog_config import hog_human, WIN_SIZE
# from ipynb.fs.full.nms import nmsf

In [2]:
clf = joblib.load('model_hog_svm.pkl')

In [3]:
def pyramid(im, scale_step=1.6, min_size=(80, 80)):
    yield im
    while True:
        h = int(im.shape[0] / scale_step)
        w = int(im.shape[1] / scale_step)
        if h < min_size[0] or w < min_size[1]:
            break
        im = cv2.resize(im, (h, w))
        yield im

In [4]:
def slide_windows(im, step_size, win_size):
    for y in range(0, im.shape[0]-step_size, step_size):
        for x in range(0, im.shape[1]-step_size, step_size):
            yield (y, x, im[y:y + win_size[0], x:x + win_size[1]])

In [5]:
img = cv2.imread("dataset/cctv_pos/1_231.png")
img = cv2.imread("sample.jpg")

In [6]:
step_size = 20
win_size = (64, 32)
bounding_boxs = np.zeros(4, np.int64)
pyramids = pyramid(img)
for c, pyr in enumerate(pyramids):
    for _, (y, x, patch) in enumerate(slide_windows(pyr, step_size, win_size)):
        if patch.shape[0] != win_size[0] or patch.shape[1] != win_size[1]:
            continue
            
        resized_patch = cv2.resize(patch, WIN_SIZE)
        descriptor = np.transpose(hog_human.compute(resized_patch))
        y_predict = clf.predict(descriptor)
        
        if (y_predict == 1):
            scale_rate = img.shape[0]/pyr.shape[0]
            bounding_boxs = np.vstack((
                bounding_boxs,
                np.array([
                    int(y*scale_rate),
                    int(x*scale_rate),
                    int((y+patch.shape[0])*scale_rate),
                    int((x+patch.shape[1])*scale_rate)]
            )))
        
        pyr_2 = pyr.copy()
        color = (255, 0, 0) if y_predict == 1 else (127,127, 127)
        drawed = cv2.rectangle(pyr_2, (x, y), (x + win_size[1], y + win_size[0]), color, 2)
        cv2.imshow("slide", drawed)
        cv2.waitKey(100)
bounding_boxs = np.delete(bounding_boxs, (0), axis=0)
cv2.destroyAllWindows()

In [7]:
def nmsf(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

In [8]:
def non_max_suppression_slow(boxes, overlapThresh):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
	# initialize the list of picked indexes
	pick = []
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
    	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list, add the index
		# value to the list of picked indexes, then initialize
		# the suppression list (i.e. indexes that will be deleted)
		# using the last index
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		suppress = [last]
        		# loop over all indexes in the indexes list
		for pos in range(0, last):
			# grab the current index
			j = idxs[pos]
			# find the largest (x, y) coordinates for the start of
			# the bounding box and the smallest (x, y) coordinates
			# for the end of the bounding box
			xx1 = max(x1[i], x1[j])
			yy1 = max(y1[i], y1[j])
			xx2 = min(x2[i], x2[j])
			yy2 = min(y2[i], y2[j])
			# compute the width and height of the bounding box
			w = max(0, xx2 - xx1 + 1)
			h = max(0, yy2 - yy1 + 1)
			# compute the ratio of overlap between the computed
			# bounding box and the bounding box in the area list
			overlap = float(w * h) / area[j]
			# if there is sufficient overlap, suppress the
			# current bounding box
			if overlap > overlapThresh:
				suppress.append(pos)
		# delete all indexes from the index list that are in the
		# suppression list
		idxs = np.delete(idxs, suppress)
	# return only the bounding boxes that were picked
	return boxes[pick]

In [9]:
ori_img = img.copy()
nms_img = img.copy()
nms_bounding_boxs = nmsf(bounding_boxs, 0.3)

In [ ]:
print(len(bounding_boxs), len(nms_bounding_boxs))
for bb in bounding_boxs:
    cv2.rectangle(ori_img, (bb[1], bb[0]), (bb[3], bb[2]), (255, 0, 0), 2)
cv2.imshow("origin", ori_img)

for nbb in nms_bounding_boxs:
    cv2.rectangle(nms_img, (nbb[1], nbb[0], nbb[3], nbb[2]), (255, 0, 0), 2)
cv2.imshow("after nms", nms_img)

cv2.waitKey()
cv2.destroyAllWindows()

59 29
